In [1]:

#hide
from fastai.vision.all import *
import fastai
# from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path


In [12]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]

In [13]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255

In [15]:
valid_3_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

In [7]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [8]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [9]:
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=256)
xb,yb = first(dl)
xb.shape,yb.shape


valid_dl = DataLoader(valid_dset, batch_size=256)

dls = DataLoaders(dl, valid_dl)
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [10]:
import torch
import fastai
learn = Learner(dls, nn.Linear(28*28,1), opt_func=fastai.optimizer.SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [11]:
learn.fit(10, lr=1)


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.637335,0.503479,0.495584,00:00
1,0.523429,0.187650,0.842983,00:00
2,0.191928,0.182256,0.837586,00:00
3,0.083938,0.106748,0.911678,00:00
4,0.044235,0.077836,0.933268,00:00
5,0.028775,0.062339,0.947988,00:00
6,0.022438,0.052730,0.955348,00:00
7,0.019634,0.046355,0.962218,00:00
8,0.018220,0.041882,0.965653,00:00
9,0.017376,0.038583,0.967125,00:00
